In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import pandas as pd
import numpy as np
import platform
from statsmodels.tsa.seasonal import seasonal_decompose

# 한글 폰트 설정
if platform.system() == 'Darwin':
    plt.rc('font', family = 'AppleGothic')
else :
    plt.rc('font', family = 'Malgun Gothic')

# 데이터 로드 (예제로 적절한 경로 설정 필요)
total_df = pd.read_csv('/mnt/data/total_df.csv')

# 날짜 형식으로 변환
total_df['Order_purchase_timestamp'] = pd.to_datetime(total_df['Order_purchase_timestamp'])
total_df['Order_delivered_customer_date'] = pd.to_datetime(total_df['Order_delivered_customer_date'])
total_df['Review_creation_date'] = pd.to_datetime(total_df['Review_creation_date'])
total_df['Review_answer_timestamp'] = pd.to_datetime(total_df['Review_answer_timestamp'])

# 필요한 변수들 생성
total_df['Delivery_speed'] = (total_df['Order_delivered_customer_date'] - total_df['Order_purchase_timestamp']).dt.days
total_df['Review_response_time'] = (total_df['Review_answer_timestamp'] - total_df['Review_creation_date']).dt.days
total_df['Number_of_products_per_order'] = total_df.groupby('Order_id')['Order_item_id'].transform('size')

# 평균 리뷰 점수 계산
total_df['Seller_average_review_score'] = total_df.groupby('Seller_id')['Review_score'].transform('mean')
total_df['Product_average_review_score'] = total_df.groupby('Product_id')['Review_score'].transform('mean')

# 임시로 사용할 변수
total_df['Loyal_Customer_Count'] = np.random.randint(1, 100, size=len(total_df))  # 임의의 값으로 채우기
total_df['Price_competitiveness'] = np.random.rand(len(total_df))  # 임의의 값으로 채우기

# MAU 계산을 위한 Year_Month 컬럼 생성
total_df['Year_Month'] = total_df['Order_purchase_timestamp'].dt.to_period('M')

# MAU 계산
mau = total_df.groupby('Year_Month')['Customer_unique_id'].nunique().reset_index()
mau.columns = ['Year_Month', 'MAU']

# 데이터와 MAU를 병합
total_df = total_df.merge(mau, on='Year_Month', how='left')

# 월별 데이터 평균 계산
monthly_data = total_df.groupby('Year_Month')[['Delivery_speed', 'Review_response_time', 'Number_of_products_per_order',
            'Seller_average_review_score', 'Product_average_review_score',
            'Loyal_Customer_Count', 'Price_competitiveness', 'MAU']].mean()

# 데이터 분해, 계절성 주기를 6개월로 설정
result = seasonal_decompose(monthly_data['MAU'], model='additive', period=6)

# 분해 결과 시각화
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 8), sharex=True)
result.observed.plot(ax=ax1, title='관측치')
result.trend.plot(ax=ax2, title='추세')
result.seasonal.plot(ax=ax3, title='계절성')
result.resid.plot(ax=ax4, title='잔차')
ax1.set_ylabel('MAU')
ax2.set_ylabel('추세')
ax3.set_ylabel('계절성')
ax4.set_ylabel('잔차')
plt.tight_layout()
plt.show()


분석 결과에 대한 자세한 설명은 각 그래프 컴포넌트—관측치(Observed), 추세(Trend), 계절성(Seasonality), 그리고 잔차(Residuals)를 기반으로 해석할 수 있습니다. 이들은 시계열 데이터를 이해하는 데 중요한 요소들입니다:

- 관측치 (Observed)
이 그래프는 실제 데이터 포인트를 시간에 따라 보여줍니다. 월별 활성 사용자 수(MAU)의 원래 관측된 값들을 시각화한 것입니다.
관측치 그래프를 통해 데이터의 전반적인 변동성과 패턴을 직관적으로 파악할 수 있으며, 주기적 또는 비정기적 변동을 확인할 수 있습니다.

- 추세 (Trend)
추세 그래프는 데이터의 장기적인 이동 방향을 나타냅니다. 이는 시간이 지남에 따라 증가하거나 감소하는 경향을 보여주며, 일시적인 변동이나 계절적 요인은 제외하고 데이터의 평활화(smoothing)된 형태를 제시합니다.
추세를 통해 비즈니스가 성장하고 있는지, 또는 어떤 문제로 인해 사용자 수가 감소하고 있는지 등의 장기적인 동향을 확인할 수 있습니다.

- 계절성 (Seasonality)
계절성 그래프는 주기적인 패턴을 보여줍니다. 이는 한 해 동안 또는 설정한 주기(본 예에서는 6개월) 동안 반복되는 패턴을 나타냅니다.
특정 시즌이나 몇 개월 간격으로 반복되는 사용자 활동의 증가 또는 감소를 파악할 수 있습니다. 예를 들어, 휴가 시즌이나 특정 이벤트 기간 동안 사용자 활동이 증가하는 경향을 확인할 수 있습니다.

- 잔차 (Residuals)
잔차 그래프는 관측치에서 추세와 계절성을 제거한 후 남은 요소를 보여줍니다. 이는 데이터에서 추세와 계절성으로 설명되지 않는 불규칙한 변동을 나타냅니다.
높은 잔차 값은 모델이 데이터의 특정 변동을 적절히 설명하지 못했다는 신호일 수 있으며, 추가적인 요인이나 외부 영향을 고려해야 할 필요가 있습니다.

- 분석의 활용
이러한 분석을 통해 얻은 정보는 다음과 같은 방법으로 활용할 수 있습니다:

- 마케팅 및 프로모션 계획: 계절성 분석 결과를 바탕으로 특정 시즌에 맞춘 마케팅 전략을 수립하고 실행할 수 있습니다.
- 재고 관리 및 리소스 배분: 사용자 활동이 증가하는 시기에 맞춰 재고를 조절하고, 필요 리소스를 효율적으로 배분할 수 있습니다.
- 사업 전략 및 계획: 추세 분석을 통해 장기적인 사업 전략을 수정하고, 성장 기회를 포착하거나 잠재적 위험을 관리할 수 있습니다.